# Transcribe the post content into a text file


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from be.data.utils import get_resource
from loguru import logger as lg
import whisper
from pathlib import Path
from pydub import AudioSegment
from whisper import Whisper

In [ ]:
posts_fol = get_resource("ig_fol") / "posts"

In [ ]:
whisper.available_models()

In [ ]:
model_type = "medium"
# model_type = "medium.en"
# model_type = "large-v3"
model = whisper.load_model(model_type)

In [ ]:
def extract_audio(video_fp: Path, audio_fp: Path, audio_format: str = "mp3") -> None:
    video_format = video_fp.suffix[1:]
    video = AudioSegment.from_file(video_fp, format=video_format)
    audio = video.set_channels(1).set_frame_rate(16000).set_sample_width(2)
    audio.export(audio_fp, format=audio_format)

In [ ]:
def extract_text(audio_fp: Path, model: Whisper) -> str:
    result = model.transcribe(str(audio_fp))
    return result["text"]

In [ ]:
for post_fol in posts_fol.iterdir():
    lg.info(post_fol)

    video_fp = post_fol / "p_video_url.mp4"
    if not video_fp.exists():
        lg.warning(f"Video not found: {video_fp}")
        continue

    transcript_fp = post_fol / f"p_transcript_{model_type}.txt"
    if transcript_fp.exists():
        lg.info(f"Transcript already exists: {transcript_fp}")
        continue

    audio_fp = post_fol / "p_audio.mp3"
    extract_audio(video_fp, audio_fp)

    text = extract_text(audio_fp, model)
    transcript_fp.write_text(text)

    break